In [2]:
#imports
import pandas as pd
#from gpt4all import GPT4All
#import gpt4all
#path = "C:\Users\Jakob\Downloads\gpt4all-falcon-q4_0.gguf"

from tqdm import tqdm
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *

import nltk
nltk.download('stopwords')
nltk.download('punkt')

import numpy as np

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Preprocessing of our data (creating smaller subset)

In [2]:
def split_file_by_lines(filename, lines_per_file,create_file):
    with open(filename, 'r', encoding='utf-8') as file: 
        count = 0
        current_file = open(create_file, 'w', encoding='utf-8') 
        
        numLine = 0
        count = 0
        for line in file:
            if(numLine > lines_per_file):
                break
            if (count % 1000) == 0:
                current_file.write(line)
                numLine += 1
            count += 1    


        current_file.close()

In [3]:
def split_file_by_lines_topic(filename, lines_per_file,create_file,topic):
    with open(filename, 'r', encoding='utf-8') as file: 
        count = 0
        current_file = open(create_file, 'w', encoding='utf-8') 
        
        numLine = 0
        count = 0
        for line in file:
            if(numLine > lines_per_file):
                break
            split_line = line.split(' ||| ')
            if topic in split_line[1]:
                current_file.write(line)
                numLine += 1
            count += 1    


        current_file.close()

get the complete titles for the queries in a dataframe

In [4]:
def txt_to_df_title(filename):
    fp = open(filename, 'r', encoding='utf-8')
    queries = []
    for entry in fp:
        query, _ = entry.split(' ||| ', 1)
        queries.append(query)
    df_queries = pd.DataFrame(queries, columns=['Query'])
    fp.close()
    return df_queries

In [5]:
def txt_to_df(filename):
    df = pd.read_csv(filename, delimiter=' \|\|\| ', engine='python', header=None, names=['Title', 'Text'])
    return df

Init the LLM

In [6]:
llm = GPT4All(model_name="gpt4all-falcon-q4_0.gguf",model_path="./model")

Generate a .txt file with the response of the llm as a document

In [9]:
def generateDocWithLLM(queries):
    fp = open("llmDocs.txt", 'a',encoding='utf-8')
    index = 0
    start = 0
    for title in queries['Title']:
        if start == 1:
            query = f'give me a short summary on {title}'
            input = query
            #print(input)
            with llm.chat_session():
                response = llm.generate(input, temp=0)
            #print(response)
            fp.write(str(title) + " ||| " + str(response) + "\n")
            # if(index == 3):
            #     break
        index += 1
        if(index % 50 == 0):
            print(index)
        if(index == 181):
            start = 1    
    fp.close()
    print("finished")  

In [10]:
#split_file_by_lines('data/stemmed.txt', 1000,'data/wikipediaSubset.txt')
#split_file_by_lines('data/raw.txt', 1000,'data/textForQueries.txt')
#df = txt_to_df('data/wikipediaSubset.txt')
#dfQ = txt_to_df_title('data/textForQueries.txt')


#split_file_by_lines_topic('data/raw.txt', 1000,'data/topic.txt',' sport ')
topic_dataframe = txt_to_df('data/topic.txt')

#display(df)
#display(dfQ)
generateDocWithLLM(topic_dataframe)

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
finished


# Preprocesses the data generated by the LLM as well as the original data

In [3]:
def file_iterator(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        for line in file:
            split = line.split("|||")
            title = split[0]
            #join in the rare case this sequenze occurs more than once
            text = ''.join(map(str, split[1:]))
            yield (title, text)

def preprocess_Data(filename, isLLMData):
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    
    df = {}

    counter = 1
    for (title, text) in file_iterator(filename):
        
        if isLLMData:
            id = "L" + str(counter)
        else:
            id = "W" + str(counter)
            
        text = title + " " +text

        #Tokenize
        text = word_tokenize(text)

        #convert words to lowercase
        text = [t.lower() for t in text]

        #remove punctuation
        text = [t for t in text if t.isalnum() or t.isspace()]

        #remove stopwords
        text = [t for t in text if t not in stopwords_english]

        #stemm
        text = [stemmer.stem(t) for t in text]
        
        df[id] = text

        counter += 1

    return df


In [82]:
data = preprocess_Data('llmDocs.txt', True)
data2 = preprocess_Data('data/topic.txt', False)
data.update(data2)

# Retriveing the first X documents using BM25. This is used for efficiently eliminating unreleveant documents. The resulting documaents are later reranked using more computaion intensive but also more acurate methods.

In [4]:
class BM25:
    def __init__(self, data):
        self.data = data
        self.k1 = 1.5
        self.b = 0.75
        self.inverted_index = self.calc_inverted_index()
        self.bm25 = self.calc_bm25_matrix()


    def calc_inverted_index(self) -> dict[str, list[str]]:

        inverted_index = {}
        
        for doc_id, words in self.data.items():
            for word in words:
                if word not in inverted_index.keys():
                    inverted_index[word] = []
                    inverted_index[word].append(doc_id)
                    
                else:
                    if doc_id not in inverted_index[word]:
                        inverted_index[word].append(doc_id)

        return inverted_index


    def calc_bm25_matrix(self):
            
        bm25 = {}     

        #construct vector for every word
        average_document_length = self.average_document_length()
        for doc_id, words in self.data.items():
            bm25[doc_id] = {}
            document_unique = np.unique(words)
            document = words
            documnet_id = doc_id
            for word in document_unique:
                idf = self.calculate_idf(len(self.inverted_index[word]))
                self.calc_bm25(word, bm25, idf, average_document_length, document, documnet_id)

        return bm25

    def calc_bm25(self, word, bm25, idf, average_document_length, document, documnet_id):
        
        term_frequency = document.count(word)
        
        numerator = idf * term_frequency * (self.k1 + 1)
        denominator = term_frequency + self.k1 * (1 - self.b + ((self.b * len(document)) / average_document_length))
        bm25[documnet_id][word] = numerator / denominator

    def average_document_length(self):
        element_counter = 0
        word_counter = 0
        for _, words in self.data.items():
            element_counter += 1
            word_counter += len(words)

        return word_counter / element_counter
    
    def calculate_idf(self, amount_documents_including_word):
        return np.log(len(self.data) / amount_documents_including_word)
    
    def preprocess_query(self, query: str):
        stemmer = PorterStemmer()
        stopwords_english = stopwords.words('english')

        #Tokenize
        query = word_tokenize(query)

        #convert words to lowercase
        query = [t.lower() for t in query]

        #remove punctuation
        query = [t for t in query if t.isalnum or t.isspace()]

        #remove stopwords
        query = [t for t in query if t not in stopwords_english]

        #stemm
        query = [stemmer.stem(t) for t in query]

        return query


    def retrieve_relevance(self, query: str, k: int) -> dict[str, float]:
        ### ADD YOUR CODE (BEGIN) ###

        query = self.preprocess_query(query)

        #filter out word not contained in any document for efficiency
        query = [word for word in query if word in self.inverted_index.keys()]

        bm25_current_query = {}

        for doc_id, words in self.data.items():
            bm25_current_query[doc_id] = []
            for word in query:
                if word in self.bm25[doc_id]:
                    bm25_current_query[doc_id].append(self.bm25[doc_id][word])
                else: 
                    bm25_current_query[doc_id].append(0)

        for key, value in bm25_current_query.items():
            bm25_current_query[key] = sum(value)

        sorted_bm25 = sorted(bm25_current_query.items(), key=lambda x:x[1], reverse=True)

        sorted_bm25 = sorted_bm25[:k]
        bm25_current_query = dict(sorted_bm25)

        return bm25_current_query


In [64]:
bm25 = BM25(data)

print(bm25.retrieve_relevance("Formula", 100))

{'L391': 7.770003009797946, 'L379': 7.31713235950478, 'W522': 7.17142430707659, 'W800': 7.017569064937496, 'W78': 6.833663961948606, 'L522': 6.723775567422254, 'W332': 6.629586529287207, 'L136': 6.581050183516847, 'W875': 6.576919123645714, 'W649': 6.5234208453362115, 'L78': 6.364878833727795, 'L800': 6.364878833727795, 'L799': 6.339601462245913, 'L474': 6.313036891030706, 'W380': 6.237563972627702, 'W377': 6.118790516979296, 'W379': 5.747785086508744, 'W391': 5.556017679334, 'L649': 5.523744662677712, 'L380': 5.466992406198085, 'L377': 5.446119317829707, 'W136': 5.263756719019585, 'L332': 5.225804182076223, 'L1': 5.190806380715372, 'W730': 5.024033151220216, 'L861': 4.958359521085838, 'L903': 4.926841466641731, 'L875': 4.834646467509674, 'W474': 4.703884797860893, 'W799': 4.094068735273467, 'W645': 4.047357747561919, 'L913': 3.831003289150197, 'W247': 3.5349828842935684, 'W783': 3.409676986828168, 'W834': 3.2937597697619356, 'W169': 3.2660015821244928, 'W870': 3.133944851642919, 'W304

---
# merge llm output and documents

In [ ]:
def merge_files(file1_path, file2_path, merged_file_path):
    try:
        with open(file1_path, 'r') as file1, open(file2_path, 'r') as file2:
            content1 = file1.read()
            content2 = file2.read()

            merged_content = content1 + content2
            
        with open(merged_file_path, 'w') as merged_file:
            merged_file.write(merged_content)

    except Exception as e:
        print(f'Error: {e}')

In [ ]:
merge_files('llmDocs.txt', './data/topic.txt', 'mixedDocs.txt')

---
# Rerank using MonoBert

In [ ]:
# download model

model_name = "castorini/monobert-large-msmarco"
model_path = "./model"

model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.save_pretrained(model_path)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.save_pretrained(model_path)

In [ ]:
model_path = "./model"

local_model = AutoModelForSequenceClassification.from_pretrained(model_path)
local_tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
def get_relevance_bert(query, documents):
    relevances = []
    for document in documents:

        inputs = tokenizer(query, document, return_tensors="pt", padding=True, truncation=True)

        # predictions    
        with torch.no_grad():
            outputs = model(**inputs)

        logits = outputs.logits

        # e.g.:
        # outputs: SequenceClassifierOutput(loss=None, logits=tensor([[-1.8822,  2.8848]]), hidden_states=None, attentions=None)
        # logits: tensor([[-1.8822,  2.8848]])

        # re-scale to [0,1] and so that sum == 1
        relevance = torch.nn.functional.softmax(logits, dim=1)
        relevances.append([document, relevance[0].tolist()[1]])

    return relevances

In [ ]:
def order_relevance(relevances):
    relevances = sorted(relevances, reverse=True, key=lambda x: x[1])
    return relevances

In [ ]:
query = "What is the capital of France?"
documents = ["Paris is the capital of France.",
             "birds like France.",
             "snakes eat gras",
             "Graz is the capital of France.",
             "France is capital paris",
             "paris is capital france",
             "France capital paris egg"]

relevances = get_relevance_bert(query, documents)
relevances = order_relevance(relevances)

for relevance in relevances:
    print(relevance)

# Full retrival pipeline using BM25 and BERT

In [19]:
class Pipeline:
    def __init__(self, amount_bm25_fetched_documents):
        data=self.getData()
        self.bm25 = BM25(data)
        self.documents = self.getCompleteFiles()
        self.amount_bm25_fetched_documents = amount_bm25_fetched_documents

    def getData(self):
        data = preprocess_Data('llmDocs.txt', True)
        data2 = preprocess_Data('data/topic.txt', False)
        data |= data2
        return data
    
    def getCompleteFiles(self):
        documents = {}

        #read LLM generated Documents
        counter = 1
        with open('llmDocs.txt', 'r', encoding='utf-8') as file:
            for line in file:
                split = line.split("|||")
                #join in the rare case this sequenze occurs more than once
                text = ''.join(map(str, split[1:]))
                documents["L"+str(counter)] = text
                counter+=1

        #read original wikipedia documents
        counter = 1
        with open('data/topic.txt', 'r', encoding='utf-8') as file:
            for line in file:
                split = line.split("|||")
                #join in the rare case this sequenze occurs more than once
                text = ''.join(map(str, split[1:]))
                documents["W"+str(counter)] = text
                counter+=1
        
        return documents
    
    def retrieve_relevant_documents_with_BERT(self, query, documents):
        relevances = get_relevance_bert(query, documents)
        relevances = order_relevance(relevances)   
        return relevance 
    
    def retrieve_relevant_documents(self, query):

        #retrieve amount_bm25_fetched_documents documents using fast bm25
        bm25_docs = self.bm25.retrieve_relevance(query, self.amount_bm25_fetched_documents)

        #fetch the non preprocessed documents retrieved by bm25
        retrieved_documents = []
        for doc_id in bm25_docs:
            retrieved_documents.append(self.documents[doc_id])

        #rerank these documents using BERT
        ranked_document_list = self.retrieve_relevant_documents_with_BERT(query, retrieved_documents)

        for doc in ranked_document_list:
            print(doc)
    

In [16]:
pip = Pipeline(100)

pip.retrieve_relevant_documents("Auto racing")

{'L1': ['auto', 'race', 'auto', 'race', 'competit', 'sport', 'involv', 'drive', 'vehicl', 'around', 'track', 'popular', 'form', 'auto', 'race', 'formula', 'one', 'featur', 'car', 'consid', 'pinnacl', 'sport', 'form', 'auto', 'race', 'includ', 'nascar', 'indycar', 'ralli', 'race', 'auto', 'race', 'requir', 'skill', 'precis', 'speed', 'compet', 'highest', 'level'], 'L2': ['america', 'nation', 'game', 'america', 'nation', 'game', 'basebal', 'play', 'two', 'team', 'nine', 'player', 'object', 'score', 'run', 'hit', 'ball', 'bat', 'run', 'around', 'four', 'base', 'order', 'game', 'origin', 'unit', 'state', '1800', 'sinc', 'becom', 'one', 'popular', 'sport', 'countri'], 'L3': ['archeri', 'archeri', 'ancient', 'sport', 'involv', 'shoot', 'arrow', 'target', 'use', 'bow', 'arrow', 'practic', 'thousand', 'year', 'origin', 'date', 'back', 'ancient', 'civil', 'egyptian', 'greek', 'roman', 'today', 'archeri', 'still', 'enjoy', 'mani', 'peopl', 'around', 'world', 'recreat', 'activ', 'competit', 'spor